In [1]:
# ==============================================
# Segmentation Pipeline for Holographic Images
# ==============================================

# --- Imports ---
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

from skimage.io import imread, imsave
from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops
from skimage.morphology import (
    binary_closing, remove_small_objects, disk, rectangle
)
from scipy.ndimage import binary_fill_holes
from skimage.transform import rotate as rotate_image
import tifffile as tiff
from tqdm import tqdm


In [2]:
# =============================
# Morphological Linking for Shape Connectivity
# =============================
def morphological_linking(mask):
    """Link elongated shapes using rotated rectangular structuring elements."""
    linked = np.zeros_like(mask)
    for angle in np.arange(0, 180, 22.5):
        selem = rotate_image(rectangle(1, 2), angle, resize=True, order=0) > 0
        dilated = binary_closing(mask, footprint=selem)
        eroded = binary_closing(dilated, footprint=selem)
        linked = np.logical_or(linked, eroded)
    return linked

In [3]:
# =============================
# Image Preprocessing and Binary Mask Creation
# =============================
def process_image(image_path, min_size=10):
    """Convert grayscale image to binary mask using morphological operations."""
    image = imread(image_path, as_gray=True)
    image = (image * 255).astype(np.uint8)

    # Thresholding and morphology
    thresh = threshold_otsu(image)
    binary = image > thresh
    closed = binary_closing(binary, footprint=disk(2))
    filled = binary_fill_holes(closed)

    # Connect elongated structures
    linked = morphological_linking(filled)
    cleaned = remove_small_objects(linked, min_size=min_size)

    return cleaned.astype(np.uint8) * 255

In [4]:
# =============================
# ROI Extraction and Padding
# =============================
def segment_and_save_rois(mask, original_image, base_name, output_dir, target_size=224):
    """Extract each connected region and save padded ROI and mask."""
    labeled = label(mask > 0)
    regions = regionprops(labeled)

    masks_dir = os.path.join(output_dir, "rois")
    crops_dir = os.path.join(output_dir, "rois_crops")
    os.makedirs(masks_dir, exist_ok=True)
    os.makedirs(crops_dir, exist_ok=True)

    for i, region in enumerate(regions):
        if region.area >= 1:
            minr, minc, maxr, maxc = region.bbox
            roi_mask = labeled[minr:maxr, minc:maxc] == region.label
            roi_image = original_image[minr:maxr, minc:maxc]

            # Center padding to fixed size
            h, w = roi_image.shape
            pad_h = max(target_size - h, 0)
            pad_w = max(target_size - w, 0)
            pad_top, pad_bottom = pad_h // 2, pad_h - pad_h // 2
            pad_left, pad_right = pad_w // 2, pad_w - pad_w // 2

            padded_image = np.pad(roi_image, ((pad_top, pad_bottom), (pad_left, pad_right)), mode='constant')
            padded_mask = np.pad(roi_mask, ((pad_top, pad_bottom), (pad_left, pad_right)), mode='constant')

            padded_image = padded_image[:target_size, :target_size]
            padded_mask = padded_mask[:target_size, :target_size]

            # Save ROI crop and mask
            mask_filename = f"{base_name}_ROI_{i:03d}_mask.tiff"
            image_filename = f"{base_name}_ROI_{i:03d}_crop.tiff"

            tiff.imwrite(os.path.join(masks_dir, mask_filename), (padded_mask.astype(np.uint8) * 255))
            inverted_crop = 255 - np.clip(padded_image * 255, 0, 255).astype(np.uint8)
            tiff.imwrite(os.path.join(crops_dir, image_filename), inverted_crop)


In [5]:
# =============================
# Batch Process Full Dataset
# =============================
def batch_process_montage_images(input_dir, output_dir, suffix="mon.tiff"):
    """Apply segmentation pipeline to all montage images in directory."""
    os.makedirs(output_dir, exist_ok=True)
    
    for filename in os.listdir(input_dir):
        if filename.endswith(suffix):
            image_path = os.path.join(input_dir, filename)
            mask = process_image(image_path)
            image = imread(image_path, as_gray=True)
            
            base_name = os.path.splitext(filename)[0]
            output_mask_path = os.path.join(output_dir, f"{base_name}_mask.tiff")
            imsave(output_mask_path, mask)
            print(f"Saved mask: {output_mask_path}")

            segment_and_save_rois(mask, image, base_name, output_dir)

In [6]:
# =============================
# Run Processing on Campaign Data
# =============================
input_dir = "/home/amtissot/Desktop/LIMNC/hologram_dataset_ML_project/campaigns/20240718/mon_files"
output_dir = "/home/amtissot/Desktop/LIMNC/hologram_dataset_ML_project/campaigns/training_data_NN_dated"
batch_process_montage_images(input_dir, output_dir)

/tmp/ipykernel_2431516/3813174140.py:8: FutureWarning: `rectangle` is deprecated since version 0.25 and will be removed in version 0.27. Use `skimage.morphology.footprint_rectangle` instead.
  selem = rotate_image(rectangle(1, 2), angle, resize=True, order=0) > 0


Saved mask: /home/amtissot/Desktop/LIMNC/hologram_dataset_ML_project/campaigns/training_data_NN_dated/009-9752-mon_mask.tiff


/tmp/ipykernel_2431516/3813174140.py:8: FutureWarning: `rectangle` is deprecated since version 0.25 and will be removed in version 0.27. Use `skimage.morphology.footprint_rectangle` instead.
  selem = rotate_image(rectangle(1, 2), angle, resize=True, order=0) > 0


Saved mask: /home/amtissot/Desktop/LIMNC/hologram_dataset_ML_project/campaigns/training_data_NN_dated/010-0790-mon_mask.tiff


/tmp/ipykernel_2431516/3813174140.py:8: FutureWarning: `rectangle` is deprecated since version 0.25 and will be removed in version 0.27. Use `skimage.morphology.footprint_rectangle` instead.
  selem = rotate_image(rectangle(1, 2), angle, resize=True, order=0) > 0
/home/amtissot/miniconda3/envs/old_project_env/lib/python3.10/site-packages/skimage/_shared/utils.py:328: UserWarning: /home/amtissot/Desktop/LIMNC/hologram_dataset_ML_project/campaigns/training_data_NN_dated/010-0947-mon_mask.tiff is a low contrast image
  return func(*args, **kwargs)
/tmp/ipykernel_2431516/3813174140.py:8: FutureWarning: `rectangle` is deprecated since version 0.25 and will be removed in version 0.27. Use `skimage.morphology.footprint_rectangle` instead.
  selem = rotate_image(rectangle(1, 2), angle, resize=True, order=0) > 0


...

/tmp/ipykernel_2431516/3813174140.py:8: FutureWarning: `rectangle` is deprecated since version 0.25 and will be removed in version 0.27. Use `skimage.morphology.footprint_rectangle` instead.
  selem = rotate_image(rectangle(1, 2), angle, resize=True, order=0) > 0


Saved mask: /home/amtissot/Desktop/LIMNC/hologram_dataset_ML_project/campaigns/training_data_NN_dated/009-9703-mon_mask.tiff


/tmp/ipykernel_2431516/3813174140.py:8: FutureWarning: `rectangle` is deprecated since version 0.25 and will be removed in version 0.27. Use `skimage.morphology.footprint_rectangle` instead.
  selem = rotate_image(rectangle(1, 2), angle, resize=True, order=0) > 0


Saved mask: /home/amtissot/Desktop/LIMNC/hologram_dataset_ML_project/campaigns/training_data_NN_dated/010-1053-mon_mask.tiff


/home/amtissot/miniconda3/envs/old_project_env/lib/python3.10/site-packages/skimage/_shared/utils.py:328: UserWarning: /home/amtissot/Desktop/LIMNC/hologram_dataset_ML_project/campaigns/training_data_NN_dated/010-1053-mon_mask.tiff is a low contrast image
  return func(*args, **kwargs)


In [7]:
# =============================
# Dataset Inspection and Label Alignment
# =============================
crop_dir_NN = os.path.join(output_dir, "rois_crops")
crop_dir = "/home/amtissot/Desktop/LIMNC/hologram_dataset_ML_project/campaigns/training_data/rois_crops"

In [8]:
# Compare image availability
tiff_files = {f for f in os.listdir(crop_dir) if f.endswith(".tiff")}
tiff_files_NN = {f for f in os.listdir(crop_dir_NN) if f.endswith(".tiff")}
print(f"Training data: {len(tiff_files)} TIFFs, NN crops: {len(tiff_files_NN)}")


Training data: 12304 TIFFs, NN crops: 4794


In [9]:
# Load and align label files
df = pd.read_csv("/home/amtissot/Desktop/LIMNC/hologram_dataset_ML_project/campaigns/roi_labels_with_features.csv")
df_2 = pd.read_csv("/home/amtissot/Desktop/LIMNC/hologram_dataset_ML_project/campaigns/roi_labels.csv")


In [10]:
# Filter to valid filenames
df_nn = df_2[df_2["filename"].isin(tiff_files_NN)][["filename", "label"]].copy()
df_nn['label'] = df_nn['label'].replace({"1": "Living", "0": "Non-living"})
print(f"Matched {len(df_nn)} image names to existing crops.")

Matched 4794 image names to existing crops.


In [ ]:

# Save aligned labels
label_out = os.path.join(output_dir, "rois_labels_nn.csv")
df_nn.to_csv(label_out, index=False)
print(f"Labels saved to {label_out}")
